In [2]:
#this file is given by instructor
#used for extracting routing demand
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

with gzip.open('xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

congestion_data = np.load('xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst


for k in range(instances.shape[0]):
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [3]:
#excluding irrelevant and meaning-less columns
df = instances.drop(columns=['name', 'id'], axis=1)
df[['xloc', 'yloc', 'cell', 'orient']]

,xloc,yloc,cell,orient
0,41984,44544,23,0
1,41984,47616,23,6
2,44160,44544,23,0
3,44160,47616,23,0
4,46336,47616,23,0
...,...,...,...,...
3947,21888,53760,42,4
3948,33664,66048,42,0
3949,23296,66048,34,0
3950,40576,44544,11,0


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error, r2_score

#set seed for repetitive test
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

#extrating the adjacency matrix
conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))

#set target y
target_values = df['routing_demand'] 

#split train/test dataset
train_size = 0.7
indices = np.arange(len(target_values))
train_indices, test_indices = train_test_split(indices, train_size=train_size, random_state=seed)

#feature engineering
scaler = StandardScaler()
features = scaler.fit_transform(df[['xloc', 'yloc', 'cell', 'orient']]) 


features = torch.FloatTensor(features)
targets = torch.FloatTensor(target_values)

input_size = features.shape[1]
output_size = 1

#build GCN model
class GCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x, A):
        for layer in self.layers:
            x = torch.relu(layer(x))
            x = torch.spmm(A, x)
        x = self.output_layer(x)
        return x


hidden_size = 20
num_layers_list = [1, 2, 3, 4, 5]
learning_rate = 0.02
num_epochs = 1000

for num_layers in num_layers_list:
    model = GCN(input_size, hidden_size, output_size, num_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        test_outputs = model(features, A)
        mse = mean_squared_error(test_outputs[test_indices].numpy(), targets[test_indices].numpy())

    print(f"Performance for {num_layers} layers - MSE: {mse}")


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 1 layers - MSE: 80.82246398925781


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 2 layers - MSE: 461.6094055175781


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 3 layers - MSE: 41996164.0


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 4 layers - MSE: 4518.43212890625


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 5 layers - MSE: 1.0377233893063818e+24


**ATTENTION GCN**

In [7]:
#copy paste from the above section except for GCN model where added an extra attention.
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error, r2_score

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))

target_values = df['routing_demand']  

train_size = 0.7
indices = np.arange(len(target_values))
train_indices, test_indices = train_test_split(indices, train_size=train_size, random_state=seed)


scaler = StandardScaler()
features = scaler.fit_transform(df[['xloc', 'yloc', 'cell', 'orient']]) 


features = torch.FloatTensor(features)
targets = torch.FloatTensor(target_values)


input_size = features.shape[1]
output_size = 1 

class AttentionGCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        #added attention layer here
        super(AttentionGCN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.attention_weights = nn.Parameter(torch.rand(num_layers))

    def forward(self, x, A):
        for layer_idx, layer in enumerate(self.layers):
            x = torch.relu(layer(x))
            attention_weight = F.softmax(self.attention_weights[layer_idx], dim=0)
            x = attention_weight * torch.spmm(A, x) + x 
        x = self.output_layer(x)
        return x

# Hyperparameters
hidden_size = 20
num_layers_list = [1, 2, 3, 4, 5]
learning_rate = 0.02
num_epochs = 1000

for num_layers in num_layers_list:
    model = AttentionGCN(input_size, hidden_size, output_size, num_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        test_outputs = model(features, A)
        mse = mean_squared_error(test_outputs[test_indices].numpy(), targets[test_indices].numpy())

    print(f"Performance for {num_layers} layers - MSE: {mse}")


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 1 layers - MSE: 80.07828521728516


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 2 layers - MSE: 6315.73779296875


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 3 layers - MSE: 214667056.0


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 4 layers - MSE: 1970.515380859375


/Applications/anaconda3/envs/cse150b/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Performance for 5 layers - MSE: 8.261854236987115e+23


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

conn = np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))

scaler = StandardScaler()
features = scaler.fit_transform(df[['xloc', 'yloc', 'cell', 'orient']])

features = torch.FloatTensor(features)
targets = torch.FloatTensor(df['routing_demand'])

input_size = features.shape[1]
output_size = 1  # Regression task

class GCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x, A):
        for layer in self.layers:
            x = torch.relu(layer(x))
            x = torch.spmm(A, x)
        x = self.output_layer(x)
        return x

hidden_size = 20
num_layers = 1
learning_rate = 0.02
num_epochs = 1000

total_samples = len(df)
mse_results = []

#for loop for loading all different folders.
for i in range(total_samples):
    train_indices = np.arange(total_samples)
    train_indices = np.delete(train_indices, i)
    test_indices = [i]

    model = GCN(input_size, hidden_size, output_size, num_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(features[train_indices], A)
        loss = criterion(outputs, targets[train_indices])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        test_outputs = model(features[test_indices], A)
        mse = mean_squared_error(test_outputs.numpy(), targets[test_indices].numpy())
        mse_results.append(mse)

average_mse = np.mean(mse_results)
print(f"Average MSE across leave-one-out experiments: {average_mse}")
